In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [2]:

pip install pgmpy


Note: you may need to restart the kernel to use updated packages.


In [3]:
cancer_model = BayesianModel([('Pollution','Cancer'),
                             ('Smoker','Cancer'),
                             ('Cancer','Xray'),
                             ('Cancer','Dyspnoea')])
print('Bayesian network nodes are:')
print("\t",cancer_model.nodes())
print('Bayesian network edges are:')
print("\t",cancer_model.edges())

cpd_poll = TabularCPD(variable='Pollution',variable_card=2,values=[[0.9],[0.1]])
cpd_smoke = TabularCPD(variable='Smoker',variable_card=2,values=[[0.3],[0.7]])
cpd_cancer = TabularCPD(variable='Cancer',variable_card=2,values=[[0.03,0.05,0.001,0.02],
                                                                 [0.97,0.95,0.999,0.98]],
                       evidence=['Smoker','Pollution'],
                       evidence_card=[2,2])
cpd_xray = TabularCPD(variable='Xray',variable_card=2,values=[[0.9,0.2],[0.1,0.8]],
                     evidence=['Cancer'],evidence_card=[2])
cpd_dysp = TabularCPD(variable='Dyspnoea',variable_card=2,values=[[0.65,0.3],[0.35,0.7]],
                     evidence=['Cancer'],evidence_card=[2])

Bayesian network nodes are:
	 ['Pollution', 'Cancer', 'Smoker', 'Xray', 'Dyspnoea']
Bayesian network edges are:
	 [('Pollution', 'Cancer'), ('Cancer', 'Xray'), ('Cancer', 'Dyspnoea'), ('Smoker', 'Cancer')]


In [4]:
cancer_model.add_cpds(cpd_poll,cpd_smoke,cpd_cancer,cpd_xray,cpd_dysp)
print('Model generated by adding cpts(cpds)')
print('Checking correctness of model:',end='')
print(cancer_model.check_model())

Model generated by adding cpts(cpds)
Checking correctness of model:True


In [5]:

print('All local depencies are as follows')
cancer_model.get_independencies()

All local depencies are as follows


(Pollution ⟂ Smoker)
(Pollution ⟂ Dyspnoea, Xray | Cancer)
(Pollution ⟂ Dyspnoea, Xray | Cancer, Smoker)
(Pollution ⟂ Xray | Dyspnoea, Cancer)
(Pollution ⟂ Dyspnoea | Cancer, Xray)
(Pollution ⟂ Xray | Dyspnoea, Cancer, Smoker)
(Pollution ⟂ Dyspnoea | Cancer, Xray, Smoker)
(Smoker ⟂ Pollution)
(Smoker ⟂ Dyspnoea, Xray | Cancer)
(Smoker ⟂ Xray | Dyspnoea, Cancer)
(Smoker ⟂ Dyspnoea, Xray | Pollution, Cancer)
(Smoker ⟂ Dyspnoea | Cancer, Xray)
(Smoker ⟂ Xray | Dyspnoea, Pollution, Cancer)
(Smoker ⟂ Dyspnoea | Pollution, Cancer, Xray)
(Xray ⟂ Dyspnoea, Pollution, Smoker | Cancer)
(Xray ⟂ Pollution, Smoker | Dyspnoea, Cancer)
(Xray ⟂ Dyspnoea, Smoker | Pollution, Cancer)
(Xray ⟂ Dyspnoea, Pollution | Cancer, Smoker)
(Xray ⟂ Smoker | Dyspnoea, Pollution, Cancer)
(Xray ⟂ Pollution | Dyspnoea, Cancer, Smoker)
(Xray ⟂ Dyspnoea | Pollution, Cancer, Smoker)
(Dyspnoea ⟂ Smoker, Pollution, Xray | Cancer)
(Dyspnoea ⟂ Pollution, Xray | Cancer, Smoker)
(Dyspnoea ⟂ Smoker, Xray | Pollution, Cancer)
(Dy

In [6]:
print('Displaying  CPDs')
print(cancer_model.get_cpds('Pollution'))
print(cancer_model.get_cpds('Smoker'))
print(cancer_model.get_cpds('Cancer'))
print(cancer_model.get_cpds('Xray'))
print(cancer_model.get_cpds('Dyspnoea'))


Displaying  CPDs
+--------------+-----+
| Pollution(0) | 0.9 |
+--------------+-----+
| Pollution(1) | 0.1 |
+--------------+-----+
+-----------+-----+
| Smoker(0) | 0.3 |
+-----------+-----+
| Smoker(1) | 0.7 |
+-----------+-----+
+-----------+--------------+--------------+--------------+--------------+
| Smoker    | Smoker(0)    | Smoker(0)    | Smoker(1)    | Smoker(1)    |
+-----------+--------------+--------------+--------------+--------------+
| Pollution | Pollution(0) | Pollution(1) | Pollution(0) | Pollution(1) |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(0) | 0.03         | 0.05         | 0.001        | 0.02         |
+-----------+--------------+--------------+--------------+--------------+
| Cancer(1) | 0.97         | 0.95         | 0.999        | 0.98         |
+-----------+--------------+--------------+--------------+--------------+
+---------+-----------+-----------+
| Cancer  | Cancer(0) | Cancer(1) |
+---------+-----------+-------

In [7]:
cancer_infer=VariableElimination(cancer_model)
print('\n Inferencing with bayesian network')
print("\n Probability of Cancer given smoker")
q=cancer_infer.query(variables=['Cancer'],evidence={'Smoker':1})
print(q)

print("\n Probability of Cancer given smoker,pollution")
q=cancer_infer.query(variables=['Cancer'],evidence={'Smoker':1,'Pollution':1})
print(q)


 Inferencing with bayesian network

 Probability of Cancer given smoker


Eliminating: Xray: 100%|██████████| 3/3 [00:00<00:00, 233.26it/s]


+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0029 |
+-----------+---------------+
| Cancer(1) |        0.9971 |
+-----------+---------------+

 Probability of Cancer given smoker,pollution


Eliminating: Xray: 100%|██████████| 2/2 [00:00<00:00, 167.08it/s]


+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0200 |
+-----------+---------------+
| Cancer(1) |        0.9800 |
+-----------+---------------+
